In [1]:
import numpy as np
import math
import tensorflow as tf

In [19]:
s_type_exp_per_atom = 3
p_type_exp_per_atom = 4
batch_size = 30
Z = np.array([1 for _ in range(2 * batch_size)])
R = np.random.random((2 * batch_size, 3))
coords = np.random.random((batch_size, 200, 3))
N = np.array([2 for _ in coords])

n_mol = tf.shape(N)[0]
mol_ids = tf.repeat(tf.range(n_mol), N)
N_electrons = tf.math.unsorted_segment_sum(Z, mol_ids, n_mol)[:, None]
# recreate grid points for each atom of the molecule separately
coords = tf.repeat(coords, N, axis=0)
# recreate atomic positions for each point on the grid
coeffs = np.random.random((Z.shape[0], 2 * s_type_exp_per_atom + 5 * p_type_exp_per_atom))
#coords = tf.transpose(coords, (0, 2, 1))
s_type_alphas = coeffs[:, :s_type_exp_per_atom] ** 2
index = s_type_exp_per_atom
s_type_cs = coeffs[:, index:index+s_type_exp_per_atom]
index += s_type_exp_per_atom
p_type_alphas = coeffs[:, index:index+p_type_exp_per_atom] ** 2
index += p_type_exp_per_atom
p_type_cs = coeffs[:, index:index+p_type_exp_per_atom]
index += p_type_exp_per_atom
v_coeffs = coeffs[:, index:]
# calculate the integrals of the orbitals for later normalization
s_type_integrals = s_type_cs * tf.math.sqrt(math.pi / s_type_alphas)
p_type_integrals = 0.5 * p_type_cs * tf.math.sqrt(math.pi ** 3 / p_type_alphas ** 5)
# create the contribution from the s_type orbitals
# calculate the norm of the s-type orbitals 
s_type_out = s_type_cs[:, :, None] * tf.exp(-1 * s_type_alphas[:, :, None] * (tf.math.reduce_sum((R[:, None, :] - coords) ** 2, axis=-1))[:, None, :])
s_type_out = tf.math.reduce_sum(s_type_out, axis=1)
s_type_out = tf.math.unsorted_segment_sum(s_type_out, mol_ids, n_mol)
# create the contribution from the p-type orbitals
v_coeffs = tf.reshape(v_coeffs, (v_coeffs.shape[0], p_type_exp_per_atom, 3))
p_type_out = p_type_cs[:, :, None] * tf.exp(-1 * p_type_alphas[:, :, None] * (tf.math.reduce_sum((R[:, None, :] - coords) ** 2, axis=-1))[:, None, :])
test = tf.math.reduce_sum((v_coeffs[:, :, None, : ] * (R[:, None, :] - coords)[:, None, :, :])**2, axis=-1)
p_type_out = p_type_out * test
p_type_out = tf.math.reduce_sum(p_type_out, axis=1)
p_type_out = tf.math.unsorted_segment_sum(p_type_out, mol_ids, n_mol)
# normalizing 
s_type_norms = tf.math.unsorted_segment_sum(s_type_integrals, mol_ids, n_mol)
s_type_norms = tf.math.reduce_sum(s_type_norms, axis=-1)
p_type_norms = tf.math.unsorted_segment_sum(p_type_integrals, mol_ids, n_mol)
p_type_norms = tf.math.reduce_sum(p_type_norms, axis=-1)
norms_sum = (s_type_norms + p_type_norms)[:, None]
#dens_out = (s_type_out + p_type_out) / (s_type_norms + p_type_norms) * tf.cast(N_electrons, dtype=tf.float32)
dens_out = (s_type_out + p_type_out) / norms_sum #* N_electrons

In [22]:
s_type_out + p_type_out

<tf.Tensor: shape=(30, 200), dtype=float64, numpy=
array([[2.51682718, 2.69503497, 2.86461325, ..., 3.05191844, 2.52469283,
        2.76587181],
       [2.89734896, 2.90040596, 2.97613405, ..., 3.02459033, 3.33838717,
        3.08620341],
       [3.50360183, 3.02264918, 3.14775571, ..., 3.5416509 , 3.05230738,
        2.94342034],
       ...,
       [3.7062355 , 3.69630771, 3.47968726, ..., 3.10832412, 3.92848541,
        4.0853649 ],
       [3.35190195, 3.00677693, 3.35477084, ..., 3.05125424, 3.205794  ,
        3.06417243],
       [3.34392153, 3.24153226, 3.19220864, ..., 3.23908457, 3.22688993,
        3.29443415]])>